In [1]:
import pandas as pd
import numpy as np
import torch

#####################
# YOU MUST WRITE YOUR STUDENT ID IN THE VARIABLE STUDENT_ID
# EXAMPLE: STUDENT_ID = "12345678"
#####################
STUDENT_ID = "20251189"


In [2]:
def save_data(df1, df2):
    # BUT you should keep the file name as "{STUDENT_ID}_simple_seq.p#.answer.csv"
    df1.to_csv(f'{STUDENT_ID}_simple_seq.p1.answer.csv')
    df2.to_csv(f'{STUDENT_ID}_simple_seq.p2.answer.csv')

In [14]:
class CustomLinear(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(CustomLinear, self).__init__()
        self.W = torch.randn(in_features, out_features, dtype=torch.float32, requires_grad=True)
        self.b = torch.randn(out_features, dtype=torch.float32, requires_grad=True)
    
    def forward(self, x):
        return x.mm(self.W) + self.b

class CustomModel(torch.nn.Module):
    def __init__(self, input_dim):
        super(CustomModel, self).__init__()
        self.layer1 = CustomLinear(input_dim, 1000)
        self.layer2 = CustomLinear(1000, 100)
        self.layer3 = CustomLinear(100, 19)
    
    def forward(self, x):
        x = self.layer1.forward(x)
        x = torch.relu(x)
        x = self.layer2.forward(x)
        x = torch.relu(x)
        x = self.layer3.forward(x)
        return x


In [4]:
column_names = [f'{i}' for i in range(1, 22)]
train = pd.read_csv('./dataset/simple_seq.train.csv', names=column_names)

def extract_last_value(row):
    non_nan_values = row.dropna().tolist() 
    return non_nan_values[-1]

true_labels = train.apply(extract_last_value, axis=1) 

def remove_last_value(row):
    non_nan_values = row.dropna().tolist()  
    non_nan_values.pop()
    return pd.Series(non_nan_values)

train = train.apply(remove_last_value, axis=1) 
train.fillna("PAD", inplace=True)

true_labels_df = pd.DataFrame(true_labels, columns=["true_label"])

print(train.head(7))
print(true_labels_df.head())
print(len(train))
print(len(true_labels_df))


     0     1     2     3     4    5    6     7     8     9    10    11    12  \
0   W25   W26   W27   W19   W28  W29  W30   W31   W32   W33  W34   W35   W36   
1   W41    W4   W42   W43   W44  W45  W46   W47   W48   W49  W50   W51   W52   
2   W55   W19   W46   W32   W32  W56  W57   W58   W59   W19  W13   W60   W19   
3   W13   W83   W32   W32   W56  W57  W13   W84   W19   W28  W85   W86   W24   
4   W87   W88   W89   W90   W32  W91  W13   W92   W93   W90  W94   W95   W24   
5   W13   W52   W32   W53   W17  W13  W96   W97   W10    W2  W98   W99   W19   
6  W122  W123  W110  W124  W125  W19  W13  W126  W127  W128  W32  W129  W130   

    13    14    15    16   17    18   19  
0  W37   W38   W39   W24  W40   PAD  PAD  
1  W53   W17   W54   W24  PAD   PAD  PAD  
2  W13   W61   W62   PAD  PAD   PAD  PAD  
3  PAD   PAD   PAD   PAD  PAD   PAD  PAD  
4  PAD   PAD   PAD   PAD  PAD   PAD  PAD  
5  W13  W100   W24   PAD  PAD   PAD  PAD  
6  W36   W13  W131  W132  W17  W133  W24  
  true_label
0 

In [5]:
column = [f'{i}' for i in range(1, 22)]
test = pd.read_csv('./dataset/simple_seq.test.csv', header=None, names=column)
test.fillna("PAD", inplace=True)
test.drop(columns=['21'], inplace=True)
print(test.head(7))

       1     2      3     4      5      6     7      8      9     10     11  \
0    W13   W81    W19  W346   W846  W1582   W70    W28  W5433    W19  W1163   
1  W5413  W111  W5414   W32    W68  W5415   W12  W2402    W19  W5438  W5439   
2  W5413  W111  W5414   W32    W68  W5415   W12   W417   W346   W336    W17   
3  W5413  W111  W5414   W32    W68  W5415   W12   W346    W32  W2833    W93   
4  W5413  W111  W5414   W32    W68  W5415   W12   W111   W346    W47   W336   
5  W5413  W111  W5414   W32    W68  W5415   W12   W346   W168  W2464  W5448   
6    W87   W31    W47   W38  W1196    W97  W627  W5449    PAD    PAD    PAD   

      12     13     14     15     16     17    18     19   20  
0  W2261    W24    PAD    PAD    PAD    PAD   PAD    PAD  PAD  
1  W5440    W12   W346   W240  W5441  W5442   W24    PAD  PAD  
2    W28  W5443    W12   W122    W47    W38  W335  W1248  W24  
3    W28  W5444  W5445    W17   W346  W5446   W24    PAD  PAD  
4   W286  W5415   W552  W5447   W641   W346   W

C:\Users\admin\AppData\Local\Temp\ipykernel_20744\2865642782.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'PAD' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test.fillna("PAD", inplace=True)


In [6]:
unique_words= set(train.values.flatten()).union({"UNK"})
vocab=sorted(list(unique_words))
word_to_index = {word: i for i, word in enumerate(vocab)}
print(len(vocab))
print(vocab[:10])

unique_labels = set(true_labels_df['true_label'].values)
label_vocab = sorted(list(unique_labels))
label_to_index = {label: i for i, label in enumerate(unique_labels)}
print(len(label_vocab))
print(label_vocab[:19])

2548
['PAD', 'UNK', 'W1', 'W10', 'W100', 'W1003', 'W1004', 'W1008', 'W1009', 'W1010']
19
['D1', 'D11', 'D12', 'D13', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D27', 'D28', 'D3', 'D32', 'D4', 'D5', 'D6', 'D7']


In [10]:
def one_hot_encode(row, vocab_size, word_to_index):
    one_hot_matrix = np.zeros((len(row), vocab_size))
    for i, word in enumerate(row):
        word = word if word in word_to_index else "UNK"
        if word != "PAD":
            one_hot_matrix[i, word_to_index[word]] = 1
    return one_hot_matrix

def one_hot_encode_label(label, label_to_index):   
    one_hot_matrix = np.zeros((1, len(label_vocab)))
    one_hot_matrix[0, label_to_index[label]] = 1
    return one_hot_matrix

In [ ]:
X_train_encoded = np.array([one_hot_encode(row, len(vocab), word_to_index) for row in train.values])
Y_train_encoded = np.array([one_hot_encode_label(label, label_to_index) for label in true_labels_df["true_label"]])

X_train = torch.tensor(X_train_encoded.reshape(X_train_encoded.shape[0], -1), dtype=torch.float32)
y_train = torch.tensor(np.argmax(Y_train_encoded, axis=1), dtype=torch.float32)

input_dim = X_train.shape[1]

model = CustomModel(input_dim)
optimizer = torch.optim.SGD([model.layer1.W, model.layer1.b,
                                model.layer2.W, model.layer2.b,
                                model.layer3.W, model.layer3.b], lr=0.1)
criterion = torch.nn.CrossEntropyLoss()

batch_size = 32
dataset = torch.utils.data.TensorDataset(X_train, y_train)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

epochs = 10
for epoch in range(epochs):
    total_loss = 0.0

    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")

RuntimeError: Expected floating point type for target with class probabilities, got Long

In [20]:
train_outputs = model(X_train)
predicted_classes = torch.argmax(train_outputs, dim=-1)
true_classes = torch.argmax(y_train, dim=-1) 

correct_predictions = (predicted_classes == true_classes).sum().item()
total_samples = true_classes.shape[0]
accuracy = correct_predictions / total_samples

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 0.44%
